### Importation des librairies

In [222]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import math
#Traitement des messages d'erreur :
warnings.filterwarnings('ignore')
import timeit
#Librairie pour gérer l'espace mémoire du PC
import gc
#Librairie pour importer des variables d'un autre notebook
import pickle

### Fonctions utilisées dans le notebook

In [223]:
# Remplacer une valeur manquantes en utilisant la médiane
def transf_med (column_to_transf):
    """Fonction pour transformer des valeurs manquantes en utilisant la médiane"""
    median = data_work[column_to_transf].median()
    data_work[column_to_transf].fillna(median, inplace=True)

### Lecture du dataset

In [224]:
path = 'Projet_7/'
data_final = pd.read_csv(path + "data_final.csv")

In [225]:
#Affichage de la totalité de la ligne 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth',3000)
display(data_final.head())

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,OWN_CAR_AGE,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_CITY_NOT_WORK_CITY,TOTALAREA_MODE,DEF_30_CNT_SOCIAL_CIRCLE,FLAG_DOCUMENT_3,AMT_REQ_CREDIT_BUREAU_YEAR,CREDIT_ACTIVE,DAYS_CREDIT,AMT_CREDIT_SUM,AMT_BALANCE,AMT_PAYMENT_CURRENT,SK_DPD_x,SK_DPD_DEF,DAYS_INSTALMENT_delay,AMT_INSTALMENT_delta,AMT_ANNUITY,AMT_CREDIT_y,AMT_DOWN_PAYMENT,DAYS_DECISION,CNT_PAYMENT,DAYS_FIRST_DRAWING,DAYS_LAST_DUE,DAYS_TERMINATION,NAME_CONTRACT_STATUS,CODE_REJECT_REASON,NAME_CLIENT_TYPE,CNT_INSTALMENT_FUTURE,SK_DPD_y
0,100002,1,Cash loans,M,0,202500.0,406597.5,Unaccompanied,working,low_level_education,not_married,0.018801,-9461,-637,NaN,other,1.0,2,2,10,0,0.0149,2.0,1,1.0,Closed,-874.00,108131.945625,NaN,NaN,NaN,NaN,-20.421053,0.0,9251.775,179055.00,0.00,-606.000000,24.000000,365243.0,-25.000000,-17.000000,Approved,XAP,New,15.000000,0.0
1,100003,0,Cash loans,F,0,270000.0,1293502.5,Family,working,high_level_education,married,0.003541,-16765,-1188,NaN,other,2.0,1,1,11,0,0.0714,0.0,1,0.0,Closed,-1400.75,254350.125000,NaN,NaN,NaN,NaN,-7.000000,0.0,56553.990,484191.00,3442.50,-1305.000000,10.000000,365243.0,-1054.333333,-1047.333333,Approved,XAP,Refreshed,4.909091,0.0
2,100004,0,Revolving loans,M,0,67500.0,135000.0,Unaccompanied,working,low_level_education,not_married,0.010032,-19046,-225,26.0,other,1.0,2,2,9,0,NaN,0.0,0,0.0,Closed,-867.00,94518.900000,NaN,NaN,NaN,NaN,-3.000000,0.0,5357.250,20106.00,4860.00,-815.000000,4.000000,365243.0,-724.000000,-714.000000,Approved,XAP,New,0.000000,0.0
3,100006,0,Cash loans,F,0,135000.0,312682.5,Unaccompanied,working,low_level_education,married,0.008019,-19005,-3039,NaN,other,2.0,2,2,17,0,NaN,0.0,1,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,-19.375000,0.0,23651.175,291695.50,34840.17,-272.444444,23.000000,365243.0,182477.500000,182481.750000,Approved,XAP,Repeater,8.650000,0.0
4,100007,0,Cash loans,M,0,121500.0,513000.0,Unaccompanied,working,low_level_education,not_married,0.028663,-19932,-3038,NaN,other,1.0,2,2,11,1,NaN,0.0,0,0.0,Closed,-1149.00,146250.000000,NaN,NaN,NaN,NaN,-6.320000,0.0,12278.805,166638.75,3390.75,-1222.833333,20.666667,365243.0,72136.200000,72143.800000,Approved,XAP,Repeater,11.666667,0.0


### Sélection des features

Nous allons décrire chacune des variables et stipuler si nous les conservons ou non.
Cela limitera aussi la quantité de données affichées ultérieurement sur notre dashboard pour une meilleure lisibilité.

- SK_ID_CURR : Identifiant du prêt --> Variable conservée
- TARGET : 0 client sans incident et 1 client avec des difficultés de remboursement --> Variable conservée
- NAME_CONTRACT_TYPE : Type de prêt contracté --> Variable conservée
- CODE_GENDER : Sexe de l'emprunteur --> Variable conservée
- CNT_CHILDREN : Nombre d'enfant de l'emprunteur --> Variable conservée (impact sur la capacité de remboursement)
- AMT_INCOME_TOTAl : Montant total des revenus --> Variable conservée (impact sur la capacité de remboursement)
- AMT_CREDIT_x : Montant total des crédits dans le bureau --> Variable conservée
- NAME_TYPE_SUITE : Indication concernant l'accompagnement de notre client --> Variable supprimée car non utile pour l'octroi d'un prêt (utile en cas de conquête de nouveaux clients)
- NAME_INCOME_TYPE = Provenance des revenus percus par le client --> Variable supprimée car non utile pour l'octroi d'un prêt (utile lors de la collecte d'épargne)
- NAME_EDUCATION_TYPE : Information surle type d'éducation du client (pré ou post BAC) --> Variable conservée
- NAME_FAMILY_STATUS : Situation familiale du client --> Variable conservée (Impact sur le nombre d'emprunteur et rentre dans le volet KYC)
- REGION_POPULATION_RELATIVE : Information sur la situtation géographique du client --> Variable non conservée au profit de la variable plus utile REGION_RATING_CLIENT
- DAYS_BIRTH : Date d'anniversaire du client --> Variable conservée car impacte l'ADE adossée au prêt (et rentre dans le cadre du KYC)
- DAYS_EMPLOYED : Nombre de jours d'emploi du client --> Variable supprimée pour éviter tout sexisme (les femmes en congés maternité pourraient avoir plus de difficultés à emprunter)
- OWN_CAR_AGE : Information sur la possession de sa proprevoirture --> Variable non conservée car non utile pour l'octroi d'un crédit
- OCCUPATION_TYPE : Type d'habitation du client --> Variable supprimée car non utile pour l'octroi d'un prêt
- CNT_FAM_MEMBERS : Nombre de membre de la famille --> Variable conservée car prend en compte le nombre de personnes totales (enfants y compris)
- REGION_RATING_CLIENT : Evaluation de la région où habite le client --> Variable conservée (le lieu d'habitation peut avoir un impact sur la fidelité et la connaissance du client)
- REGION_RATING_CLIENT_W_CITY : Evaluation de la région où vit le client en tenant compte de la ville --> Variable supprimée au profit de la variable plus utile REGION_RATING_CLIENT
- HOUR_APPR_PROCESS_START : Heure où le client a demandé le prêt --> Variable supprimée car non utile pour l'octroi d'un prêt
- REG_CITY_NOT_WORK_CITY : Clients qui n'habitent pas dans la ville où ils travaillent --> Variable supprimée au profit de la variable plus utile REGION_RATING_CLIENT
- TOTALAREA_MODE : Information sur la taille des habitations des clients --> Variable supprimée car non utile pour l'octroi d'un prêt
- DEF_30_CNT_SOCIAL_CIRCLE : Observations sociales sur les clients --> Variable supprimée car non utile pour l'octroi d'un prêt (Cette information prise en compte pourrait être considérée comme de la descrimination)
- FLAG_DOCUMENT_3 : Document à fournir à la banque --> Variable supprimée car non utile pour l'octroi d'un crédit (ce document est nécessaire pour toute étude et donc sera obligatoire en cas d'acceptation mais aussi de refus lors du montage du dossier)
- AMT_REQ_CREDIT_BUREAU_YEAR : Nombre de demandes de renseignements au bureau de crédit concernant le client un jour par an --> Variable conservée car elle permet de juger du relationnel avec la banque (un contact régulier augmente la fidelité du client)
- CREDIT_ACTIVE : Statut des crédits déclarés par le bureau de crédit --> Variable supprimée au profit de la variable plus utile AMT_CREDIT_x
- DAYS_CREDIT : Nombre de jours depuis la dernière demande de crédit réalisée par le client --> Variable supprimée car non utile vue la volatilité des taux d'emprunts et les changements de situation pro et perso des clients
- AMT_CREDIT_SUM : Montant total des prêts --> Variable supprimée au profit de AMT_CREDIT_x
- AMT_BALANCE : Montant des soldes clients --> Varaible supprimée car non utile lors de l'octroi d'un crédit
- AMT_PAYMENT_CURRENT : Montants des paiments courants --> Variable supprimée car non utilisée pour calculer le taux d'entement d'un prêt
- SK_DPD_x : jours de retard au cours du mois du crédit précédent --> Variable supprimée car un retard d'une seule échéance ne justifie pas du refus de l'octroi d'un crédit (situation exceptionnelle peut arriver à tout moment en revanche si nous avions le nombre de retard de paiement cela aurait été plus pertinent)
- SK_DPD_DEF : Jours de retard au cours d'une période donnée du crédit précédent --> Variable supprimée car un retard d'une seule échéance ne justifie pas du refus de l'octroi d'un crédit (situation exceptionnelle peut arriver à tout moment en revanche si nous avions le nombre de retard de paiement cela aurait été plus pertinent) 
- DAYS_INSTALMENT_delay : le temps entre la date du commencement des paiments et la date du début des échelonnements du crédit --> Variable conservée
- AMT_INSTALMENT_delta : La différence entre les montants des paiemants de crédits et les montants des échelonnements --> Variable conservée
- AMT_ANNUITY : Montant des annuités --> Variable conservée
- AMT_CREDIT_y : Montant du crédit de la demande précédente --> Variable supprimée au profit de la variable AMT_CREDIT_x 
- AMT_DOWN_PAYMENT : Acompte sur la demande précédente --> Variable supprimée car non utile pour l'octroi d'un crédit
- DAYS_DECISION : Nombre de jours pour la validation de la dernière offre de prêt via l'application --> Variable supprimée car non utile pour l'octroi d'un crédit
- CNT_PAYMENT : Durée du crédit précédent souscrit pour la demand de crédit --> Variable supprimée car non utile pour l'octroi d'un crédit
- DAYS_FIRST_DRAWING : Date du premier déblocage du précédent prêt via l'application --> Variable supprimée car non utile pour l'octroi d'un crédit
- DAYS_LAST_DUE : Derniere date de la demande de prêt --> Variable supprimée car non utile pour l'octroi d'un crédit
- DAYS_TERMINATION : Date de fin de la demande précédente --> Variable supprimée car non utile pour l'octroi d'un crédit
- NAME_CONTRACT_STATUS : État du contrat au cours du mois --> Variable supprimée car nous somme au moment de l'octroi du crédit et cette variable donne une information sur un prêt déjà accepté et en cours ou non de déblocage
- CODE_REJECT_REASON : Motif de rejet de la précédente demande de prêt --> Variable supprimée pour éviter tout à priori sur la demande de prêt
- NAME_CLIENT_TYPE : Information sur le statut du client (client ou non client) lors de la dernière demande de prêt --> Variable supprimée car non utile lors de l'octroi d'un crédit (elle pourra être nécessaire pour évaluer le taux de fidelité du client)
- CNT_INSTALMENT_FUTURE : Acomptes restant à payer sur le crédit précédent --> Variable supprimée au profit de la variable AMT_CREDIT plus utile.
- SK_DPD_y : Jours de retard au cours du mois du crédit précédent --> Variable supprimée car un retard d'une seule échéance ne justifie pas du refus de l'octroi d'un crédit (situation exceptionnelle peut arriver à tout moment en revanche si nous avions le nombre de retard de paiement cela aurait été plus pertinent)

In [226]:
#Création du dataset avec les variables conservées :
data_work = data_final.copy()
del data_final
list_to_delete = ["NAME_TYPE_SUITE", "NAME_INCOME_TYPE", "REGION_POPULATION_RELATIVE", "OWN_CAR_AGE", 
                  "OCCUPATION_TYPE", "REGION_RATING_CLIENT_W_CITY", "HOUR_APPR_PROCESS_START",
                  "REG_CITY_NOT_WORK_CITY", "TOTALAREA_MODE", "FLAG_DOCUMENT_3", "CREDIT_ACTIVE",
                  "AMT_BALANCE", "DAYS_EMPLOYED", "AMT_PAYMENT_CURRENT", "SK_DPD_x", "SK_DPD_DEF",
                  "AMT_CREDIT_y", "AMT_DOWN_PAYMENT", "DAYS_DECISION", "CNT_PAYMENT", 
                  "DAYS_FIRST_DRAWING", "DAYS_LAST_DUE", "DAYS_TERMINATION", "NAME_CONTRACT_STATUS",
                  "CODE_REJECT_REASON", "NAME_CLIENT_TYPE", "CNT_INSTALMENT_FUTURE", "SK_DPD_y",
                  "DEF_30_CNT_SOCIAL_CIRCLE", "AMT_CREDIT_SUM"]
data_work.drop(list_to_delete, axis =1, inplace = True)
display(data_work.head())
print(data_work.shape)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,DAYS_BIRTH,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_CREDIT,DAYS_INSTALMENT_delay,AMT_INSTALMENT_delta,AMT_ANNUITY
0,100002,1,Cash loans,M,0,202500.0,406597.5,low_level_education,not_married,-9461,1.0,2,1.0,-874.00,-20.421053,0.0,9251.775
1,100003,0,Cash loans,F,0,270000.0,1293502.5,high_level_education,married,-16765,2.0,1,0.0,-1400.75,-7.000000,0.0,56553.990
2,100004,0,Revolving loans,M,0,67500.0,135000.0,low_level_education,not_married,-19046,1.0,2,0.0,-867.00,-3.000000,0.0,5357.250
3,100006,0,Cash loans,F,0,135000.0,312682.5,low_level_education,married,-19005,2.0,2,NaN,NaN,-19.375000,0.0,23651.175
4,100007,0,Cash loans,M,0,121500.0,513000.0,low_level_education,not_married,-19932,1.0,2,0.0,-1149.00,-6.320000,0.0,12278.805


(307511, 17)


In [227]:
#On conserve la colonne des identifiants dans un dataset différent pour pas que cela impact le modèle
data_id = data_work["SK_ID_CURR"]
#On sépare la variable recherchée TARGET dans un autre dataset
data_target = data_work["TARGET"]
#On supprime ensuite ces deux variables du dataset original
data_work.drop(["SK_ID_CURR", "TARGET"], axis = 1, inplace = True)
display(data_id.head())
print(data_id.shape)
display(data_target.head())
print(data_target.shape)
display(data_work.head())
print(data_work.shape)
#Export des fichiers data_id et data_target en CSV
pd.DataFrame(data_target).to_csv(path + 'data_target.csv',index=False)
pd.DataFrame(data_id).to_csv(path + 'data_id.csv',index=False)

0    100002
1    100003
2    100004
3    100006
4    100007
Name: SK_ID_CURR, dtype: int64

(307511,)


0    1
1    0
2    0
3    0
4    0
Name: TARGET, dtype: int64

(307511,)


,NAME_CONTRACT_TYPE,CODE_GENDER,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,DAYS_BIRTH,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_CREDIT,DAYS_INSTALMENT_delay,AMT_INSTALMENT_delta,AMT_ANNUITY
0,Cash loans,M,0,202500.0,406597.5,low_level_education,not_married,-9461,1.0,2,1.0,-874.00,-20.421053,0.0,9251.775
1,Cash loans,F,0,270000.0,1293502.5,high_level_education,married,-16765,2.0,1,0.0,-1400.75,-7.000000,0.0,56553.990
2,Revolving loans,M,0,67500.0,135000.0,low_level_education,not_married,-19046,1.0,2,0.0,-867.00,-3.000000,0.0,5357.250
3,Cash loans,F,0,135000.0,312682.5,low_level_education,married,-19005,2.0,2,NaN,NaN,-19.375000,0.0,23651.175
4,Cash loans,M,0,121500.0,513000.0,low_level_education,not_married,-19932,1.0,2,0.0,-1149.00,-6.320000,0.0,12278.805


(307511, 15)


Nous avons donc x_train qui est représenté par data_work et y_train par data_target.
Nous allons maintenant créer nos datasets de train et de test et les préparer pour utiliser notre modèle

In [228]:
data_work.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 15 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   NAME_CONTRACT_TYPE          307511 non-null  object 
 1   CODE_GENDER                 307511 non-null  object 
 2   CNT_CHILDREN                307511 non-null  int64  
 3   AMT_INCOME_TOTAL            307511 non-null  float64
 4   AMT_CREDIT_x                307511 non-null  float64
 5   NAME_EDUCATION_TYPE         307511 non-null  object 
 6   NAME_FAMILY_STATUS          307511 non-null  object 
 7   DAYS_BIRTH                  307511 non-null  int64  
 8   CNT_FAM_MEMBERS             307509 non-null  float64
 9   REGION_RATING_CLIENT        307511 non-null  int64  
 10  AMT_REQ_CREDIT_BUREAU_YEAR  265992 non-null  float64
 11  DAYS_CREDIT                 263491 non-null  float64
 12  DAYS_INSTALMENT_delay       251957 non-null  float64
 13  AMT_INSTALMENT

In [229]:
#On renomme les colonnes pour une meilleure compréhension 
data_work.rename(columns = {"NAME_CONTRACT_TYPE": "Type_de_pret", "CODE_GENDER": "Genre",
                            "CNT_CHILDREN": "Nb_enfants", "AMT_INCOME_TOTAL": "Montant_des_revenus",
                            "AMT_CREDIT_x": "Montants_du_pret", "NAME_EDUCATION_TYPE" : "Niveau_d_etudes",
                            "NAME_FAMILY_STATUS" : "Regime_matrimonial", "DAYS_BIRTH" : "Age",
                            "CNT_FAM_MEMBERS" : "Nb_membre_famille", "REGION_RATING_CLIENT" : "Note_region_client",
                            "AMT_REQ_CREDIT_BUREAU_YEAR" : "Nb_demande_client", "DAYS_CREDIT" : "Nb_jours_credits",
                            "DAYS_INSTALMENT_delay": "Delai_anticipation_pret", "AMT_INSTALMENT_delta": "Montant_anticipation_pret",
                            "AMT_ANNUITY": "Montant_des_annuites"}, inplace = True)
display(data_work.head())

,Type_de_pret,Genre,Nb_enfants,Montant_des_revenus,Montants_du_pret,Niveau_d_etudes,Regime_matrimonial,Age,Nb_membre_famille,Note_region_client,Nb_demande_client,Nb_jours_credits,Delai_anticipation_pret,Montant_anticipation_pret,Montant_des_annuites
0,Cash loans,M,0,202500.0,406597.5,low_level_education,not_married,-9461,1.0,2,1.0,-874.00,-20.421053,0.0,9251.775
1,Cash loans,F,0,270000.0,1293502.5,high_level_education,married,-16765,2.0,1,0.0,-1400.75,-7.000000,0.0,56553.990
2,Revolving loans,M,0,67500.0,135000.0,low_level_education,not_married,-19046,1.0,2,0.0,-867.00,-3.000000,0.0,5357.250
3,Cash loans,F,0,135000.0,312682.5,low_level_education,married,-19005,2.0,2,NaN,NaN,-19.375000,0.0,23651.175
4,Cash loans,M,0,121500.0,513000.0,low_level_education,not_married,-19932,1.0,2,0.0,-1149.00,-6.320000,0.0,12278.805


In [230]:
#Remplacement des valeurs des colonnes pour une meilleure compréhension
#Nous prendrons la médiane lorsque des valeurs numériques seront manquantes ou impossibles
print("Voici les valeurs possibles de la colonne Type_de_pret : ", data_work['Type_de_pret'].unique())
data_work['Type_de_pret'] = data_work['Type_de_pret'].replace('Cash loans', 'Prêts de trésorerie')
data_work['Type_de_pret'] = data_work['Type_de_pret'].replace('Revolving loans', 'Prêts renouvelables')
print("Voici le résultat après traitement : ", data_work['Type_de_pret'].unique())
print("Voici les valeurs possibles dans la colonne Genre : ", data_work['Genre'].unique())
print("Voici les valeurs possibles de la colonne Nb_enfants : ", data_work['Nb_enfants'].unique())
#Nous avons jusqu'à 19 enfants dans notre étude
#On enlève les chiffres après la virgule pour les montant des revenus
print("Le montant maximum des revenus est de : ", data_work['Montant_des_revenus'].max())
data_work['Montant_des_revenus'] = data_work['Montant_des_revenus'].astype(np.int64)
#On enlève les chiffres après la virgule pour les montants de prêt
print("Le montant de prêt maximum est de : ", data_work['Montants_du_pret'].max())
data_work['Montants_du_pret'] = data_work['Montants_du_pret'].astype(np.int64)
print("Voici les valeurs possibles dans la colonne Niveau_d_etudes : ", data_work['Niveau_d_etudes'].unique())
data_work['Niveau_d_etudes'] = data_work['Niveau_d_etudes'].replace('low_level_education', '3 à 4')
data_work['Niveau_d_etudes'] = data_work['Niveau_d_etudes'].replace('high_level_education', '5 à 8')
print("Voici le résultat après traitement : ", data_work['Niveau_d_etudes'].unique())
#On renomme les valeurs possibles pour le régime matrimonial
print("Voici les valeurs possibles dans la colonne Regime_matrimonial : ", data_work['Regime_matrimonial'].unique())
data_work['Regime_matrimonial'] = data_work['Regime_matrimonial'].replace('not_married', 'célibataire')
data_work['Regime_matrimonial'] = data_work['Regime_matrimonial'].replace('married', 'marié(e)')
data_work['Regime_matrimonial'] = data_work['Regime_matrimonial'].replace('other', 'autre')
print("Voici le résultat après traitement : ", data_work['Regime_matrimonial'].unique())
#Traitement de la date de naissance en repassant la valeur en positif et en mettant plutôt l'âge du client
data_work['Age'] = ((data_work['Age'])/-360).astype(np.int64)
print("Voici l'âge du plus jeune client : ", data_work['Age'].min())
print("Voici l'âge du plus vieux client : ", data_work['Age'].max())
#On remplace les Nan par la médiane
transf_med('Nb_membre_famille')
data_work['Nb_membre_famille'] = data_work['Nb_membre_famille'].astype(np.int64)
print("Voici les valeurs possibles de la colonne Nb_membre_famille : ", data_work['Nb_membre_famille'].unique())
#On remplace les Nan par la médiane
transf_med("Nb_membre_famille")
print("Voici le résultat après traitement : ", data_work['Nb_membre_famille'].unique())
print("Voici les valeurs possibles de la colonne Note_region_client : ", data_work['Note_region_client'].unique())
#Nous avons 3 type de région avec le 3 comme meilleur région et le 1 la moins bonne
#On enlève les chiffres après la virgule des nombres de colonnes client et on remplace les Nan par 0 n'ayant pas connaissance
#du nombre de demande client
print("Voici les valeurs possibles dans la colonne Nb_demande_client : ", data_work['Nb_demande_client'].unique())
#On remplace les Nan par la médiane
transf_med("Nb_demande_client")
data_work['Nb_demande_client'] = data_work['Nb_demande_client'].astype(np.int64)
print("Vérification du traitement de la variable Nb_demande_client : ", data_work['Nb_demande_client'].unique())
#Nous avons de 0 à 25 demandes réalisées par le client
transf_med('Nb_jours_credits')
data_work['Nb_jours_credits'] = (data_work['Nb_jours_credits'].astype(np.int64))*(-1)
print("Voici le nombre de jours de crédits minimum : ", data_work['Nb_jours_credits'].min())
print("Voici le nombre de jours de crédits maximum : ", data_work['Nb_jours_credits'].max())
#On remplace les Nan par la médiane
transf_med('Delai_anticipation_pret')
data_work['Delai_anticipation_pret'] = (data_work['Delai_anticipation_pret'].astype(np.int64))*(-1)
print("Voici le délai d anticipation minimum : ", data_work['Delai_anticipation_pret'].min())
print("Voici le délai d anticipation maximum : ", data_work['Delai_anticipation_pret'].max())
#On remplace les Nan par la médiane
transf_med('Montant_anticipation_pret')
data_work['Montant_anticipation_pret'] = data_work['Montant_anticipation_pret'].astype(np.int64)
print("Voici le montant d anticipation minimum : ", data_work['Montant_anticipation_pret'].min())
print("Voici le montant d anticipation maximum : ", data_work['Montant_anticipation_pret'].max())
#Les montants/délais d'anticipation négatifs sont dûs à des retard de paiement et les montants/délais positifs 
#nous informent que le client commence à rembourser son prêt.
#On remplace les Nan par la médiane
transf_med('Montant_des_annuites')
data_work['Montant_des_annuites'] = data_work['Montant_des_annuites'].astype(np.int64)
print("Voici le montant des annuités minimum : ", data_work['Montant_des_annuites'].min())
print("Voici le montant des annuités maximum : ", data_work['Montant_des_annuites'].max())

display(data_work.head())

Voici les valeurs possibles de la colonne Type_de_pret :  ['Cash loans' 'Revolving loans']
Voici le résultat après traitement :  ['Prêts de trésorerie' 'Prêts renouvelables']
Voici les valeurs possibles dans la colonne Genre :  ['M' 'F' 'XNA']
Voici les valeurs possibles de la colonne Nb_enfants :  [ 0  1  2  3  4  7  5  6  8  9 11 12 10 19 14]
Le montant maximum des revenus est de :  117000000.0
Le montant de prêt maximum est de :  4050000.0
Voici les valeurs possibles dans la colonne Niveau_d_etudes :  ['low_level_education' 'high_level_education']
Voici le résultat après traitement :  ['3 à 4' '5 à 8']
Voici les valeurs possibles dans la colonne Regime_matrimonial :  ['not_married' 'married' 'other']
Voici le résultat après traitement :  ['célibataire' 'marié(e)' 'autre']
Voici l'âge du plus jeune client :  20
Voici l'âge du plus vieux client :  70
Voici les valeurs possibles de la colonne Nb_membre_famille :  [ 1  2  3  4  5  6  9  7  8 10 13 14 12 20 15 16 11]
Voici le résultat ap

,Type_de_pret,Genre,Nb_enfants,Montant_des_revenus,Montants_du_pret,Niveau_d_etudes,Regime_matrimonial,Age,Nb_membre_famille,Note_region_client,Nb_demande_client,Nb_jours_credits,Delai_anticipation_pret,Montant_anticipation_pret,Montant_des_annuites
0,Prêts de trésorerie,M,0,202500,406597,3 à 4,célibataire,26,1,2,1,874,20,0,9251
1,Prêts de trésorerie,F,0,270000,1293502,5 à 8,marié(e),46,2,1,0,1400,7,0,56553
2,Prêts renouvelables,M,0,67500,135000,3 à 4,célibataire,52,1,2,0,867,3,0,5357
3,Prêts de trésorerie,F,0,135000,312682,3 à 4,marié(e),52,2,2,1,1050,19,0,23651
4,Prêts de trésorerie,M,0,121500,513000,3 à 4,célibataire,55,1,2,0,1149,6,0,12278


In [231]:
#Réorganisation des colonnes
data_work = data_work[['Type_de_pret', 'Genre', 'Age', 'Niveau_d_etudes', 'Regime_matrimonial', 
                       'Nb_enfants', 'Nb_membre_famille', 'Montant_des_revenus', 
                       'Note_region_client', 'Nb_demande_client', 'Montants_du_pret',
                       'Montant_des_annuites', 'Nb_jours_credits', 'Montant_anticipation_pret',
                       'Delai_anticipation_pret',
                        ]]
display(data_work.head())

,Type_de_pret,Genre,Age,Niveau_d_etudes,Regime_matrimonial,Nb_enfants,Nb_membre_famille,Montant_des_revenus,Note_region_client,Nb_demande_client,Montants_du_pret,Montant_des_annuites,Nb_jours_credits,Montant_anticipation_pret,Delai_anticipation_pret
0,Prêts de trésorerie,M,26,3 à 4,célibataire,0,1,202500,2,1,406597,9251,874,0,20
1,Prêts de trésorerie,F,46,5 à 8,marié(e),0,2,270000,1,0,1293502,56553,1400,0,7
2,Prêts renouvelables,M,52,3 à 4,célibataire,0,1,67500,2,0,135000,5357,867,0,3
3,Prêts de trésorerie,F,52,3 à 4,marié(e),0,2,135000,2,1,312682,23651,1050,0,19
4,Prêts de trésorerie,M,55,3 à 4,célibataire,0,1,121500,2,0,513000,12278,1149,0,6


In [232]:
data_work.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 15 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   Type_de_pret               307511 non-null  object
 1   Genre                      307511 non-null  object
 2   Age                        307511 non-null  int64 
 3   Niveau_d_etudes            307511 non-null  object
 4   Regime_matrimonial         307511 non-null  object
 5   Nb_enfants                 307511 non-null  int64 
 6   Nb_membre_famille          307511 non-null  int64 
 7   Montant_des_revenus        307511 non-null  int64 
 8   Note_region_client         307511 non-null  int64 
 9   Nb_demande_client          307511 non-null  int64 
 10  Montants_du_pret           307511 non-null  int64 
 11  Montant_des_annuites       307511 non-null  int64 
 12  Nb_jours_credits           307511 non-null  int64 
 13  Montant_anticipation_pret  307511 non-null  

In [233]:
#Export du dataset data_work en CSV
pd.DataFrame(data_work).to_csv(path + 'data_work.csv',index=False)

Ce notebook a du être découpé en deux parties suite à des incidents techniques lors du lancement
Fin de la première partie du notebook P7_Part6_Features_And_Final_Model_Part1.